We estimate the FLOPs used by KataGo via iterating through all KataGo training games.

Training games are obtained from:
1. https://katagotraining.org/games/
2. https://katagoarchive.org/kata1/traininggames/index.html
3. https://katagoarchive.org/g170/selfplay/index.html

### Load libraries

In [1]:
import contextlib
import os
import pathlib
import sys
import warnings

import git.repo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ptflops
import requests
import thop
import torch
from bs4 import BeautifulSoup
from torch import nn
from tqdm.auto import tqdm

# We need to import some libraries from upstream KataGo.
# KataGo doesn't have a setup.py, so we modify sys.path instead (kinda hacky).
GIT_ROOT = pathlib.Path(
    str(git.repo.Repo(".", search_parent_directories=True).working_tree_dir)
)
sys.path.append(str(GIT_ROOT / "submodules/KataGo/python"))

import modelconfigs
from model_pytorch import Model

### Count distributed training games

In [2]:
def parse_table(table):
    rows = table.find_all("tr")
    header = [th.text.strip() for th in rows[0].find_all("th")]
    data = [[td.text.strip() for td in row.find_all("td")] for row in rows[1:]]
    return pd.DataFrame(data, columns=header)


def parse_network_name(name: str) -> dict[str, int | str]:
    net_size = name.split("-")[1]
    if net_size.endswith("x2"):
        net_size = net_size.rstrip("x2")

    return dict(
        net_size=net_size,
        steps=int(name.split("-")[2].lstrip("s")),
        rows=int(name.split("-")[3].lstrip("d")),
    )


# Pull data from https://katagotraining.org/games/
r = requests.get("https://katagotraining.org/games/")
soup = BeautifulSoup(r.content, "html.parser")

df = parse_table(soup.find_all("table")[0]).query("Network != 'kata1-random'")

df["n_train"] = (
    df["Training Games"].str.replace(",", "").str.extract(r"(\d+)").astype(int)
)
df["n_rating"] = (
    df["Rating Games"].str.replace(",", "").str.extract(r"(\d+)").astype(int)
)

df_net = pd.DataFrame([parse_network_name(x) for x in df["Network"]])
df = pd.concat([df, df_net], axis=1)

df


,Network,Training Games,Rating Games,n_train,n_rating,net_size,steps,rows
0,kata1-b60c320-s7047906048-d3140270330,"9,863 training games",44 rating games,9863,44,b60c320,7047906048,3140270330
1,kata1-b60c320-s7035181568-d3137113165,"64,096 training games",515 rating games,64096,515,b60c320,7035181568,3137113165
2,kata1-b60c320-s7022594048-d3133696060,"65,419 training games",764 rating games,65419,764,b60c320,7022594048,3133696060
3,kata1-b60c320-s7010139136-d3130207575,"71,911 training games",828 rating games,71911,828,b60c320,7010139136,3130207575
4,kata1-b60c320-s6997549568-d3127123425,"62,929 training games",824 rating games,62929,824,b60c320,6997549568,3127123425
...,...,...,...,...,...,...,...,...
559,kata1-b6c96-s5214720-d1690538,0 training games,"1,816 rating games",0,1816,b6c96,5214720,1690538
560,kata1-b6c96-s4136960-d1510003,0 training games,"1,941 rating games",0,1941,b6c96,4136960,1510003
561,kata1-b6c96-s1995008-d1329786,0 training games,"3,139 rating games",0,3139,b6c96,1995008,1329786
562,kata1-b6c96-s938496-d1208807,0 training games,"5,552 rating games",0,5552,b6c96,938496,1208807


In [3]:
STRONGEST_CONFIDENTLY_RATED_NET = "kata1-b40c256-s11840935168-d2898845681"
LAST_G170_NET = "kata1-b40c256x2-s5095420928-d1229425124"

STRONGEST_CONFIDENTLY_RATED_ROWS = int(
    parse_network_name(STRONGEST_CONFIDENTLY_RATED_NET)["rows"]
)
LAST_G170_ROWS = int(parse_network_name(LAST_G170_NET)["rows"])

cdf = df.query(f"rows <= {STRONGEST_CONFIDENTLY_RATED_ROWS} & net_size != 'b60c320'")
print("Distributed training games to reach strongest confidently rated net:")
print(cdf["n_train"].sum(), "training games")
print(cdf["n_rating"].sum(), "rating games")

rows_per_game_train = (STRONGEST_CONFIDENTLY_RATED_ROWS - LAST_G170_ROWS) / cdf[
    "n_train"
].sum()
print()
print("Rows / training game for distributed training:", rows_per_game_train)

print()
print("Total training games assuming constant row/game ratio:")
print(STRONGEST_CONFIDENTLY_RATED_ROWS / rows_per_game_train, "training games")

Distributed training games to reach strongest confidently rated net:
28279595 training games
1438771 rating games

Rows / training game for distributed training: 59.032689718505516

Total training games assuming constant row/game ratio:
49105769.95259751 training games
